# Pre procesamiento

En este *notebook* se aplicará el pre-procesamiento a cada comentario de reddit. El resultado se guardará en un archivo que es similar al archivo origen, con la única diferencia que el comentario estará conformado por *strings* procesados.

Se realizan los siguientes pre-procesamientos:
1. Eliminación de emails, *quotes*, *emojis* y url en cada comentario
2. Eliminación de *stop words*
3. Eliminación de palabras no alfanuméricas
4. Conversión de los lemas a minúscula
5. Se agrega bigramas y trigramas
6. Lematización utilizando Spacy
7. Solo se consideran palabras cuyo *part-of-speech* son un nombre propio, un sustantivo o un pronombre. [Ver *Universal POS tags*](https://universaldependencies.org/docs/u/pos/)

### Fuente

- [Twitter Topic Modeling](https://towardsdatascience.com/twitter-topic-modeling-e0e3315b12e2)


In [2]:
import pickle
import warnings

import pandas as pd
import spacy, gensim
from spacy.tokenizer import Tokenizer

from preprocessing_utils import give_emoji_free_text, url_free_text, \
email_free_text, quotes_free_text, get_lemmas, tokenize

warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

TEXT_FILE_READ = 'docs/reddit_data.csv'  # Text to be processed
TEXT_SAVE_FILE = 'docs/preprocessing_reddit_data.csv'
FILENAME_PICKLE = "docs/tmpreddit.pickle"

In [4]:
nlp = spacy.load('es_core_news_lg', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed
tmpreddit = pd.read_csv(TEXT_FILE_READ)

In [5]:
# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

# Apply `call_emoji_free` which calls the function to remove all emoji's
tmpreddit['emails_free'] = tmpreddit['body'].apply(email_free_text)

#Create a new column with url free tweets
tmpreddit['quotes_free'] = tmpreddit['emails_free'].apply(quotes_free_text)

# Apply `call_emoji_free` which calls the function to remove all emoji's
tmpreddit['emoji_free'] = tmpreddit['quotes_free'].apply(call_emoji_free)

#Create a new column with url free tweets
tmpreddit['url_free'] = tmpreddit['emoji_free'].apply(url_free_text)

#print(tmpreddit[:1])

In [6]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

tokens = []

for doc in tokenizer.pipe(tmpreddit['url_free'], batch_size=500):
    doc_tokens = []
    for token in doc:
        if token.text.lower() not in nlp.Defaults.stop_words:
            doc_tokens.append(token.text.lower())
    tokens.append(doc_tokens)

# Makes tokens column
tmpreddit['tokens'] = tokens


In [7]:
# Build the bigram and trigram model
bigram = gensim.models.Phrases(tmpreddit['tokens'], min_count=10, threshold=100)
trigram = gensim.models.Phrases(bigram[tmpreddit['tokens']], threshold=100)

# Faster way to get a sentence clubbed as a bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


In [8]:
# See trigram example
print(trigram_mod[bigram_mod[tmpreddit['tokens'][3]]])


['probado', 'resultado', 'sellar', 'boca', 'inodoro', 'y', 'tirar', 'cadena..', 'desarrollo:', 'tenia', 'tapa', 'apoyaba', 'totalidad', 'circunferencia', 'boca', 'inodoro,', 'ponía', 'hoja', 'diario', 'tapando', 'boca,', 'cerraba', 'tapa', 'y', 'sentaba', 'tapa,', 'bordes', 'boca', 'sellaran', 'papel.', 'tirar', 'cadena,', 'aire', 'adentro', 'copa', 'salir', 'ningun', 'y,', 'entrada', 'agua,', 'genera', 'presion', 'esté', 'tapando', 'caño', 'agarra', 'viaje', 'nomas.', 'cambié', 'tapa', 'cubre', 'todo,', 'papel', 'diario,', 'pongo', 'almohadón,', 'resultado,', 'único', 'lavar', 'almohadón', 'despues..', 'gracias', 'venir', 'a', 'tedtalk.']


In [9]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [10]:
# Form Bigrams
data_words_bigrams = make_bigrams(tmpreddit['tokens'])

In [12]:
# Make tokens a string again
tmpreddit['tokens_back_to_text'] = [' '.join(map(str, l)) for l in data_words_bigrams]

tmpreddit['lemmas'] = tmpreddit['tokens_back_to_text'].apply(get_lemmas)


In [13]:
# Make lemmas a string again
tmpreddit['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in tmpreddit['lemmas']]



In [14]:
tmpreddit = tmpreddit.drop_duplicates(subset=['lemmas_back_to_text'], keep='first', inplace=False)

In [15]:
# Apply tokenizer
tmpreddit['lemma_tokens'] = tmpreddit['lemmas_back_to_text'].apply(tokenize)

In [16]:
reddit = tmpreddit
reddit['body_preprocessing'] = tmpreddit['lemmas_back_to_text']
reddit['body_no_lemmas'] = tmpreddit['tokens_back_to_text']
reddit.pop('emails_free')
reddit.pop('quotes_free')
reddit.pop('emoji_free')
reddit.pop('url_free')
reddit.pop('tokens')
reddit.pop('tokens_back_to_text')
reddit.pop('lemmas')
reddit.pop('lemmas_back_to_text')


0                           tapastar baño tirar balde agua
1                         sopapa master tapón teñir medias
2                          sopapa tira agua caliente balde
3        probado resultado sellar boca inodoro tirar ca...
4        cobrar mantenimiento carajo kjjjjjjjjj viviria...
                               ...                        
43488                        lista comprometer contradecir
43495                 reir demas mem humor curar depresion
43736            pibe x200b x200b solucion inflacion x200b
43932                                   flaco ver x200b it
44078    compratir carabina andar local computacion x20...
Name: lemmas_back_to_text, Length: 27782, dtype: object

In [19]:
print(reddit)

       score       id       flair  comms_num  \
0          1  hfw14mt  Discusion🧐          1   
1          1  hfw41eh  Discusion🧐          0   
2          1  hfw1ao2  Discusion🧐          0   
3          1  hfw3jof  Discusion🧐          2   
4          1  hfw6v4i  Discusion🧐          0   
...      ...      ...         ...        ...   
43488      1  hhcqvbr  Política🏛️          1   
43495      7  hhcu8ne       IAMA🙋          0   
43736    305  hhb96tt     Video📽️          1   
43932      1  hhbjbiu    Noticia📰          0   
44078      1  hheds7c  Discusion🧐          0   

                                                    body comment_parent_id  \
0      todo para decir que tapaste el baño. tira un b...            q44kw3   
1      sopapa primero master, si hay tapón te vas a t...           hfw14mt   
2      Usas la sopapa, o tiras agua caliente con un b...            q44kw3   
3      Lo que he probado que siempre me dio resultado...            q44kw3   
4      Estas cobrando por dar man

In [ ]:
reddit.to_csv(TEXT_SAVE_FILE, index=False)

fileObj = open(FILENAME_PICKLE, 'wb')
pickle.dump(tmpreddit, fileObj)
fileObj.close()

In [ ]:
print(reddit)